<a href="https://colab.research.google.com/github/GabyLeon10100/MujeresDigitales/blob/main/Taller_semana_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de pagos a préstamos

## Introducción

**Contexto comercial.** Como analista de un banco se le entrega un conjunto de datos con información de los clientes y si pagaron el crédito o no. Su jefe le pida que entrene algunos modelos para responder la pregunta: si un usuario incurrirá en incumplimiento o no. El algoritmo para predecir la probabilidad de incumplimiento es el núcleo del negocio.

**Problema comercial.** Su tarea es **crear un modelo de clasificación para determinar si un usuario incumplirá con su préstamo o no**.

**Contexto analítico.** Como científico de datos, ahora debe concentrarse en perfeccionar el algoritmo de clasificación. Debe verificar cuantitativamente que el algoritmo funcionará para datos futuros. Además, debe expandir su conjunto de herramientas para no depender únicamente de la regresión logística, ya que tiene algunas deficiencias que se manejan mejor con otros tipos de modelos. 

## Evaluar los diferentes modelos

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.model_selection import RandomizedSearchCV

In [4]:
#Traemos el archivo "pokemon_data.csv"
from google.colab import files
uploaded = files.upload()

Saving Lending_club_cleaned_2.csv to Lending_club_cleaned_2.csv


In [5]:
df = pd.read_csv("Lending_club_cleaned_2.csv")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df.head()

,loan_status,annual_inc,verification_status,emp_length,home_ownership,int_rate,loan_amnt,purpose,term,grade
0,Fully Paid,24000.0,Verified,10+ years,RENT,10.65%,5000,credit_card,36 months,B
1,Charged Off,30000.0,Source Verified,< 1 year,RENT,15.27%,2500,car,60 months,C
2,Fully Paid,12252.0,Not Verified,10+ years,RENT,15.96%,2400,small_business,36 months,C
3,Fully Paid,49200.0,Source Verified,10+ years,RENT,13.49%,10000,other,36 months,C
4,Fully Paid,80000.0,Source Verified,1 year,RENT,12.69%,3000,other,60 months,B


## Ejercicio 1

Realice una estandarización de los datos númericos usando el promedio y la desviación estándar esto es: $(datos - promedio) / desviación\ estándar$, para las columnas ```[loan_amnt, int_rate, annual_inc]```. Y para las variables categóricas use one hot encoding ```[home_ownership, purpose, grade, loan_status]```.

Tip: Para las variables categóricas recuerde que puede usar la función de pandas: get_dummies

In [8]:
# Para el campo int_rate se debe cambiar el porcentaje por nada ya que necesitamos manejar los valores numericos
df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)

In [14]:
x = pd.concat([(df['loan_amnt'] - df['loan_amnt'].mean()) / df['loan_amnt'].std(),
            (df['annual_inc'] - df['annual_inc'].mean()) / df['annual_inc'].std(),
            (df['int_rate'] - df['int_rate'].mean()) / df['int_rate'].std(),
             pd.get_dummies(df['home_ownership'], drop_first=True),
             pd.get_dummies(df['purpose'], drop_first=True),
             pd.get_dummies(df['grade'], drop_first=True)
            ], axis = 1)
x.head()

,loan_amnt,annual_inc,int_rate,OTHER,OWN,RENT,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,B,C,D,E,F,G
0,-0.843862,-0.710290,-0.378954,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,-1.178519,-0.616866,0.863143,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,-1.191906,-0.893213,1.048651,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,-0.174546,-0.317912,0.384586,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,-1.111588,0.161661,0.169505,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [17]:
y = pd.get_dummies(df['loan_status'], drop_first=True)
y

,Fully Paid
0,1
1,0
2,1
3,1
4,1
...,...
38700,1
38701,1
38702,1
38703,1


## Ejercicio 1.1:

Realice una separación de los datos: entrenamiento 80% y prueba 20%

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

## Ejercicio 2:

Entrene 3 modelos para predecir si un cliente pagará el crédito o no:
1. Regresión logística
2. Árbol aleatorio (random forest)
3. KNN (vecinos mas cercanos)

In [22]:
# Regresión logística
logistica = LogisticRegression()
logistica.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [24]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

## Ejercicio 3:

Compare los tres modelos anteriores usando la métrica precisión. Recuerde que la fórmula es: $VP/(VP+FP)$

Donde:


VP: Verdaderos positivos

FP: Falsos positivos

In [26]:
# Escriba aquí su código
preclog = logistica.predict(x_test)
precision_score(y_test, preclog)

0.8631660631143301

In [27]:
precar = rf.predict(x_test)
precision_score(y_test, precar)

0.8661427810335642

In [28]:
prcnei = knn.predict(x_test)
precision_score(y_test, prcnei)

0.8666755996248158

GABRIELA LEÓN GONZÁLEZ

## Ejercicio 4: (Opcional)

Usando la función `RandomizedSearchCV` busque el número de vecinos mas cercanos (k) que mejor prediga el conjunto de datos. Use los siguientes valores: `n_neighbors: entre 1 y 8`, `cross validation: 10` y ``scoring: 'precision'``

In [ ]:
# Escriba aquí su código


## Ejercicio 5: (Opcional)

Extraiga el valor de precisión del modelo optimizado en el ejercicio 4.

In [ ]:
# Escriba aquí su código
